In [140]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [141]:
from gpt.gpt import GPT 

In [142]:
# Get a GPT object and do a simple run using the DC GUN template example
G = GPT(input_file='/Users/colwyngulliford/Documents/GitHub/lume-gpt/templates/dcgun/gpt.in',
       workdir='/Users/colwyngulliford/Documents/GitHub/lume-gpt/devel/test')

In [143]:
# Run GPT with some new settings:
G.set_variables({'gun_voltage':375,'sol_1_current':-1})
G.run()

zavg = [tout['z'].mean() for tout in G.tout] 
xstd = [1000*tout['x'].std() for tout in G.tout] 
ystd = [1000*tout['y'].std() for tout in G.tout] 
avgKE = [0.51*(tout['G'].mean()-1) for tout in G.tout] 

plt.figure()
plt.plot(zavg,xstd, zavg,ystd)
plt.xlabel('z (m)')
plt.ylabel('$\sigma_{x,y}$ (mm)')

plt.figure()
plt.plot(zavg,avgKE)
plt.xlabel('z (m)')
plt.ylabel('KE (MeV)')
plt.show()

Run Aborted: Command '['/Users/colwyngulliford/Documents/GitHub/xgpt/gpt321MacOSX/bin/gpt', '-v -o', 'gpt.out.gdf', 'gpt.in']' returned non-zero exit status 1.


AttributeError: 'GPT' object has no attribute 'tout'